<a href="https://colab.research.google.com/github/Sonalkumari05/tic_tac_toe_game/blob/main/minmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import json

In [24]:
def is_winner(board, player):
    # Check rows, columns, and diagonals
    for i in range(3):
        # Check each row for a win
        if all(cell == player for cell in board[i]):
            return True

        # Check each column for a win
        if all(board[j][i] == player for j in range(3)):
            return True

    # Check the main diagonal
    if all(board[i][i] == player for i in range(3)):
        return True

    # Check the other diagonal
    if all(board[i][2 - i] == player for i in range(3)):
        return True

    # No winning configuration found
    return False


In [25]:
def is_full(board):
    # Check if all cells in the board are not empty
    return all(cell != ' ' for row in board for cell in row)


In [26]:
def get_empty_cells(board):
    # Generate a list of coordinates for empty cells
    return [(i, j) for i in range(3) for j in range(3) if board[i][j] == ' ']


In [27]:
def minimax(board, depth, maximizing_player, minimax_values):
    state_key = str(board)

    # If the state has already been evaluated, return the stored value
    if state_key in minimax_values:
        return minimax_values[state_key]

    # Check if the current state is a terminal state (win, lose, or draw)
    if is_winner(board, 'X'):
        return -1
    elif is_winner(board, 'O'):
        return 1
    elif is_full(board):
        return 0

    # If it's the maximizing player's turn (AI, 'O' player)
    if maximizing_player:
        max_eval = float('-inf')
        for i, j in get_empty_cells(board):
            # Simulate making a move
            board[i][j] = 'O'
            # Recursively call minimax for the next state
            eval = minimax(board, depth + 1, False, minimax_values)
            # Undo the move
            board[i][j] = ' '
            # Update the maximum evaluation value
            max_eval = max(max_eval, eval)
        # Store the evaluated value for the current state
        minimax_values[state_key] = max_eval
        return max_eval
    else:
        # If it's the minimizing player's turn (human player, 'X' player)
        min_eval = float('inf')
        for i, j in get_empty_cells(board):
            # Simulate making a move
            board[i][j] = 'X'
            # Recursively call minimax for the next state
            eval = minimax(board, depth + 1, True, minimax_values)
            # Undo the move
            board[i][j] = ' '
            # Update the minimum evaluation value
            min_eval = min(min_eval, eval)
        # Store the evaluated value for the current state
        minimax_values[state_key] = min_eval
        return min_eval





In [28]:
def save_minimax_values():
    # Create the initial empty Tic-Tac-Toe board
    initial_board = [[' ' for _ in range(3)] for _ in range(3)]
    # Initialize the dictionary to store Mini-Max values
    minimax_values = {}

    # Run the Mini-Max algorithm to compute values for each state
    minimax(initial_board, 0, False, minimax_values)

    # Save the computed Mini-Max values to a JSON file
    with open('minimax_value.json', 'w') as file:
        json.dump(minimax_values, file, indent=2)




In [29]:
save_minimax_values()

In [30]:
def print_board(board):
    # Iterate through each row in the board
    for row in board:
        # Print each cell in the row separated by '|', creating a visual representation of a row
        print('|'.join(row))
        # Print a line of dashes to separate rows for better visual clarity
        print('-' * 5)


In [31]:
def load_minimax_values():
    # Open the 'minimax_value.json' file in read mode
    with open('minimax_value.json', 'r') as file:
        # Use json.load to parse the JSON content and return the loaded data
        return json.load(file)


In [32]:
def get_best_move(board, minimax_values):
    # Get a list of coordinates for empty cells on the board
    empty_cells = get_empty_cells(board)
    best_moves = []

    # If there are no empty cells, return None (no valid moves)
    if not empty_cells:
        return None

    # Iterate through each empty cell
    for i, j in empty_cells:
        # Simulate making a move
        board[i][j] = 'O'
        # Get the state key for the current board configuration
        state_key = str(board)
        # Get the Mini-Max value for the current state
        minimax_value = minimax_values.get(state_key, 0)
        # Append the move and its associated Mini-Max value to the list
        best_moves.append(((i, j), minimax_value))
        # Undo the move
        board[i][j] = ' '

    # Sort the list of moves based on the Mini-Max values in descending order
    best_moves.sort(key=lambda x: x[1], reverse=True)

    # Return the coordinates of the best move (the one with the highest Mini-Max value)
    return best_moves[0][0]


In [33]:
def play_game():
    # Initialize an empty Tic-Tac-Toe board
    board = [[' ' for _ in range(3)] for _ in range(3)]
    # Load precomputed Mini-Max values from the file
    minimax_values = load_minimax_values()

    while True:
        # Display the current state of the board
        print_board(board)

        # Get the human player's move
        row = int(input("Enter the row (0, 1, or 2): "))
        col = int(input("Enter the column (0, 1, or 2): "))

        # Check if the chosen cell is empty
        if board[row][col] == ' ':
            # Make the human player's move
            board[row][col] = 'X'

            # Check if the human player wins
            if is_winner(board, 'X'):
                print_board(board)
                print("Congratulations! You win!")
                break
            # Check if the board is full, resulting in a tie
            elif is_full(board):
                print_board(board)
                print("It's a tie!")
                break

            # AI's turn
            print("AI's turn:")

            # Get the AI's best move using the Mini-Max values
            ai_move = get_best_move(board, minimax_values)

            # Make the AI's move
            if ai_move is not None:
                print(f"AI plays at {ai_move}")
                board[ai_move[0]][ai_move[1]] = 'O'

                # Check if the AI wins
                if is_winner(board, 'O'):
                    print_board(board)
                    print("AI wins! Better luck next time.")
                    break
                # Check if the board is full, resulting in a tie
                elif is_full(board):
                    print_board(board)
                    print("It's a tie!")
                    break
            else:
                print("It's a tie!")
                break
        else:
            # Display an error message for an invalid move
            print("Invalid move. Try again.")

# Run the Tic-Tac-Toe game
play_game()


 | | 
-----
 | | 
-----
 | | 
-----
Enter the row (0, 1, or 2): 1
Enter the column (0, 1, or 2): 1
AI's turn:
AI plays at (0, 0)
O| | 
-----
 |X| 
-----
 | | 
-----
Enter the row (0, 1, or 2): 2
Enter the column (0, 1, or 2): 2
AI's turn:
AI plays at (0, 2)
O| |O
-----
 |X| 
-----
 | |X
-----
Enter the row (0, 1, or 2): 1
Enter the column (0, 1, or 2): 1
Invalid move. Try again.
O| |O
-----
 |X| 
-----
 | |X
-----
Enter the row (0, 1, or 2): 1
Enter the column (0, 1, or 2): 0
AI's turn:
AI plays at (0, 1)
O|O|O
-----
X|X| 
-----
 | |X
-----
AI wins! Better luck next time.
